# 신규 구매 정보 추출

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

data_path = '/Users/seonghoonjung/.kaggle/data/santander-product-recommendation'

pd.set_option('display.max_columns', None)

In [6]:
trn = pd.read_csv(f'{data_path}/train_ver2.csv')

/Users/seonghoonjung/anaconda3/envs/ai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# 제품 변수
prods = trn.columns[24:].tolist()

# 날짜를 편의상 숫자로 변환 
def date_to_int(str_date):
    Y, M, D = [ int(a)  for a in str_date.strip().split('-')]  # 2015-01-28 => 2015, 1, 28
    int_date = (int(Y) - 2015)*12 + int(M)  # months from 2015
    return int_date
trn['int_date'] = trn['fecha_dato'].map(date_to_int).astype(np.int8)

# lag-1 데이터 생성
trn_lag = trn.copy()
trn_lag['int_date'] += 1
trn_lag.columns = [col+'_prev' if col not in ['ncodpers', 'int_date'] else col for col in trn.columns]

# 원본 데이터와 lag 데이터를 합친다. (저번 달의 제품 정보가 삽입된다.)
df_trn = trn.merge(trn_lag, on=['ncodpers', 'int_date'], how='left')

del trn, trn_lag

# 저번 달의 제품 정보가 존재하지 않을 경우 0 대처 
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
    
# 신규 구매 변수 계산 (저번달에는 없다가, 이번달에는 보유한 것)
for prod in prods:
    padd = prod + '_add'
    prev = prod + '_prev'
    df_trn[padd] = ((df_trn[prod] == 1) & (df_trn[prev] == 0)).astype(np.int8)

# 신규 구매 변수만을 추출하여 label에 저장 
add_cols = [prod + '_add' for prod in prods]
labels = df_trn[add_cols].copy()
labels.columns = prods
labels.to_csv(f'{data_path}/labels.csv', index=False) 